## 1. Group Information

暂时空着

## 2. Contribution Statement

Each group member contributes to the project in different roles, such as requirements clarification, architecture and database design, backend implementation, frontend implementation, testing, documentation, and presentation. In the final submitted version, this section should provide a clear mapping of **who** did **what**. For example, you can summarise major responsibilities and contributions of each member in a short paragraph or a small table (e.g., "Alice: database schema and analytics queries; Bob: Flask APIs and concurrency tests; Charlie: Vue frontend and usability evaluation; ...").

The following table summarises the main contributions of each team member for this project:

| 成员 (Member) | 主要贡献 (Main Contributions) |
| --- | --- |
| Zhou Linquan | 项目框架设计，前后端 |
| Li Yuhang | 前端，写报告 |
| Tan Cong | 后端业务层实现，DB 实现 |
| Huang Kaisi | 后端 API 实现，DB 实现 |
| Xia Jin | 测试，写报告 |

## 3. Project Overview

### 1. Context and Problem Statement

This project targets **exhibition and performance spaces**, such as public art venues, cultural centres, community theatres, library event areas, and science exhibition halls. These venues frequently host various **public, non-commercial events**, including talks, screenings, workshops, small performances, and themed exhibitions.

In practice, event registration and check-in in such venues often suffer from several issues:

- **Fragmented tools and siloed data**  
  Many venues rely on a mixture of paper forms, Excel sheets, and ad-hoc online forms. Data for different events and sessions lives in different files or platforms. After events end, answering questions such as “which audience segments attended which types of events?” requires significant manual consolidation and cleaning.

- **Unreliable capacity management**  
  For popular events or small venues, staff need to enforce a strict capacity limit. Without a unified system, manual counting is error-prone: it is easy to over-book seats or fail to detect duplicated registrations.  
  A typical problem in public venues is that some people **register early to “reserve” seats but never show up**, while others who genuinely want to attend cannot register because “the session is already full”.

- **Inefficient on-site check-in**  
  Staff often check paper lists or manually search exported spreadsheets to verify attendees. This leads to long queues, mis-checks, and an overall poor experience for both staff and visitors.

- **Limited analytics and service optimisation**  
  Venues may want to know:  
  - Which types of events attract which audience groups?  
  - How do “members”, “media”, and general visitors differ in attendance behaviour?  
  - What are the typical attendance rates for different time slots or themes?  
  Due to fragmented data, these questions are hard to answer in a systematic way.

Our system aims to address these issues in a **public, non-commercial** context, focusing on **fair allocation of limited seats** and **data-driven service improvement**, rather than on ticket sales or payment processing.

---

### 2. System Goals and Positioning

Given the above context, we design and implement an **event registration and check-in system** for exhibition/performance spaces with the following goals:

1. **Unified management of multiple events and sessions**  
   Model events that may span multiple days, each event consisting of multiple sessions (time slots). Each session has its own start/end time and capacity.

2. **Strict capacity control and fair allocation without payments**  
   The system **does not handle any ticket prices, payments, or refunds**. All events are assumed to be **free to attend**.  
   Instead of financial logic, the system focuses on:
   - Per-session capacity limits;
   - Waiting lists for popular sessions;
   - A no-show penalty mechanism.  
   Together, these aim to make seat allocation more orderly and fair in a public-resource setting.

3. **Improving resource utilisation and fairness with waiting lists and no-show penalties**  
   - When a session is full, the system maintains a **waiting list**, typically in FIFO order. If a registered user cancels, waiting users can be automatically or semi-automatically promoted to `registered`.  
   - For users who repeatedly register but do not show up (no-show), the system tracks their behaviour. If a user accumulates **three no-shows within 30 days**, their ability to register for new sessions is **blocked for one month**.  
   These mechanisms are not about “punishing” users financially; they aim to:
   - Reduce wasted seats caused by “reserve but never come” behaviour;
   - Give genuinely interested visitors a better chance of obtaining a seat;
   - Improve overall utilisation and fairness of limited public resources.

4. **Efficient QR-code based check-in**  
   After successful registration, the system generates a digital “ticket” containing a QR code. The QR payload is a JSON object encoding the user and registration information.  
   On-site staff use a dedicated **check-in panel** to scan the QR codes, verify the registration, and mark the attendee as checked-in. This reduces check-in bottlenecks, minimises manual search, and avoids concurrency problems at the check-in stage.

5. **Operator-facing analytics and export for further analysis**  
   The system provides an analytics panel for staff to:
   - View daily totals of registrations and check-ins across all sessions;
   - Analyse participation and attendance by event type or tag;
   - Analyse participation by audience group (e.g., VIP, Media, Member, General public);  
   It also supports exporting per-event or per-session registration and attendance data to CSV, enabling further analysis and visualisation in external tools.

---

### 3. Target Users and Roles

The application distinguishes three main roles:

1. **Visitor**  
   - Browse upcoming events and sessions of the venue;
   - Filter events by date, tag, or event type;
   - Register for specific sessions and cancel registrations;
   - Obtain a ticket with a QR code and use it for on-site check-in;
   - Subject to the no-show policy: if they accumulate three no-shows within the last 30 days, their registration capability is blocked for one month.

2. **Staff**  
   - Use the back-office interface to create/edit/archive events and sessions;
   - Manage event tags and audience group information;
   - View registration and attendance lists for events and sessions;
   - **Force-add attendees** (e.g., VIPs, media guests, on-site registrations) regardless of normal capacity and waiting list constraints;
   - Operate the check-in panel to scan QR codes and complete attendee check-in.

3. **Admin**  
   - A unique or very limited high-privilege account;
   - Manages staff accounts, global configuration, and system integrity;
   - Can perform low-level maintenance and corrective operations when necessary.

---

### 4. Core Features and Characteristics

The system provides several core functional modules:

1. **Event and session management**  
   - An event (e.g., a week-long exhibition) can contain multiple sessions (e.g., different days or time slots).  
   - Each session has its own:
     - start and end time;
     - capacity;
     - `current_registered` counter;
     - `waiting_list_limit`;
     - status (open/closed).  
   - Events have a lifecycle:  
     - `draft`: being edited, not visible to visitors;  
     - `published`: visible and open for registration (subject to session status);  
     - `closed`: registration ended, sessions may still be checked-in or viewed;  
     - `archived`: historical events kept for data analysis.

2. **Registration and waiting list**  
   When a visitor attempts to register for a session:

   - If `current_registered < capacity`:
     - A new REGISTRATION is created with status `registered`;
     - The session’s `current_registered` is incremented.
   - Otherwise (session is full):
     - If a waiting list is enabled and not full, the user is added as `waiting`;
     - Otherwise, the registration is rejected as “full”.

   When a registered user cancels:

   - Their REGISTRATION status is updated to `cancelled`;
   - The session’s `current_registered` is decremented;
   - If there are waiting users, the earliest waiting user can be promoted to `registered`, and `current_registered` is incremented.

   This mechanism ensures that, for popular sessions, seats freed by cancellations can be reassigned to waiting visitors instead of being wasted.

3. **QR-code based check-in and the check-in panel**  
   - After a successful registration, the system generates a ticket-style image with an embedded QR code.
   - The QR code encodes a JSON payload containing identifiers such as user ID, session ID, and registration ID.
   - On-site staff use the check-in panel to scan the QR code:
     - The panel sends a check-in request to the backend;
     - The backend validates the registration, marks `checkin_time`, and returns basic visitor information (name, audience group, etc.);
     - The panel displays this information to staff for quick verification.  
   Because visitors physically queue to scan one by one, check-in concurrency is relatively low, and the operation is designed to be idempotent (repeated scans do not double-count check-ins).

4. **No-show tracking and registration blocking**  
   - A user is considered a **no-show** for a session if:
     - their registration status is `registered`;
     - the session has ended (end_time < now);
     - `checkin_time` is still NULL.
   - Each time a user tries to register for a new session, the system:
     - Checks whether their `blocked_until` timestamp is in the future; if so, the registration is rejected;
     - Otherwise, counts no-shows in the last 30 days. If the count is ≥ 3:
       - The system sets `blocked_until` to 30 days from now;
       - The current registration is rejected with an explanation.  
   This mechanism is designed to protect the fairness and efficiency of public resource allocation by discouraging repeated “reserve but never come” behaviour.

5. **Analytics and CSV export**  
   - The analytics panel offers aggregate views, such as:
     - Total number of registrations today across all sessions;
     - Total number of check-ins today;
     - Participation and attendance rates for specific event types or tags;
     - Participation breakdown by audience group (e.g., VIP, Media, Member, General public).  
   - For each event or session, staff can export a CSV file containing registration and attendance records, enabling further analysis in external tools (e.g., spreadsheets, BI dashboards).

---

### 5. Technical Focus: Concurrency and Data Integrity

Although the expected concurrency level is moderate (e.g., around 100 concurrent users at peak), the project deliberately focuses on **transaction management and concurrency control**:

1. **Concurrent registration and capacity enforcement**  
   - The relational schema separates EVENT, EVENT_SESSION, and REGISTRATION, with `current_registered` maintained per session.
   - During registration, the system uses:
     - Database transactions;
     - Row-level locking (e.g., `SELECT ... FOR UPDATE` on the session row).  
   - This ensures that under concurrent registration workloads:
     - The number of `registered` users never exceeds the configured capacity;
     - The `current_registered` counter remains consistent with the underlying registrations.

   We plan micro-benchmarks that simulate multiple threads registering simultaneously for a session with limited capacity, to empirically verify that no over-booking occurs.

2. **Efficient analytical queries**  
   - Appropriate indexes are defined on key columns (event type, session time, status, group_name, etc.).
   - For the dashboard, we experiment with **serial vs. application-level parallel execution** of multiple analytic queries (e.g., using a thread pool), and compare response times to illustrate the impact of parallel query execution.

---

### 6. Open-source and Reusability

A key design decision is to treat this system as an **open-source, reusable infrastructure component** for public and cultural venues, rather than a closed, venue-specific solution or a commercial ticketing platform.

- **No binding to payment providers or commercial ticketing platforms**  
  The system does not manage any payment flow and does not depend on third-party ticketing services. Events are assumed to be free by default. This makes the system particularly suitable as a piece of infrastructure for public institutions and non-profit venues, without conflicting with any existing commercial ticketing workflows they may already use.

- **Deployable in diverse venues**  
  The system can be deployed in various contexts with minimal customisation:
  - Art galleries, cultural centres, community theatres;
  - Library event spaces, science museums;
  - University venues hosting free public events.  
  Venues can adapt the UI theme or audience group definitions (e.g., VIP / Media / Member / General) while keeping the core database and logic intact.

- **Prepared for further extension and research**  
  The database schema and interfaces are designed to be clear and modular, so that future work can extend the system with:
  - More advanced role-based access control;
  - Recommendation features based on users’ historical registrations;
  - Deeper integration with external analytics or visualisation platforms.

In summary, this project provides a practical, open-source registration and check-in system tailored to public exhibition and performance spaces, while also serving as a case study in database modelling, transaction management, concurrency control, and analytical query optimisation in a non-commercial setting.

## 4. Technical Design

### 4.1 Software Architecture

We adopt a **monolithic web application** architecture with a **front-end / back-end separation** and a **layered, MVC-style design** on the backend.

- **Monolithic backend service**  
  All server-side functionality is implemented inside a single Flask application. Given the size and scope of this project, a monolithic architecture is more appropriate than microservices:
  - It keeps deployment and configuration simple.
  - It avoids the overhead of distributed transactions, inter-service communication, and complex DevOps.
  - It is sufficient to demonstrate database design, transaction management, and concurrency control, which are the main technical focuses of this project.

- **Front-end / back-end separation**  
  The system is logically split into:
  - A **Vue-based frontend** running in the browser, responsible for all user interactions and page rendering.
  - A **Flask-based backend** exposing RESTful APIs for data access and business operations.
  All communication between the frontend and backend is performed via HTTP(S) with JSON payloads.

- **Layered, MVC-style backend design**  
  Inside the monolithic backend, we follow an MVC-ish layered structure:

  1. **API / Controller layer**  
     - Flask routes and blueprints act as controllers.  
     - Responsibilities:
       - Receive HTTP requests from the Vue frontend.
       - Parse and validate request parameters and JSON bodies.
       - Invoke the appropriate service methods.
       - Return JSON responses and HTTP status codes.

  2. **Service (business logic) layer**  
     - Encapsulates core domain logic such as:
       - Registration and cancellation flows.
       - Waiting list management and promotion.
       - No-show detection and automatic blocking.
       - Analytics queries and CSV export.
     - This layer defines clear transactional boundaries and coordinates database operations through the repository layer.

  3. **Data Access / Repository layer**  
     - Uses SQLAlchemy ORM models to represent database tables.
     - Provides repository functions for CRUD operations and complex queries.
     - Shields the service layer from raw SQL details and ensures consistent session/transaction handling.

  4. **Database layer**  
     - A MySQL (InnoDB) database stores all persistent data and enforces integrity via primary keys, foreign keys, unique constraints, and check constraints.
     - Transaction support and row-level locking are used to implement safe concurrent registration (e.g., using `SELECT ... FOR UPDATE` on session rows).

This combination—monolithic backend, Vue frontend, RESTful communication, and a layered MVC-style backend—provides a clear separation of concerns while remaining practical for a course project.

---

### 4.2 Technology Stack

The main technologies used in the system are summarised below.

- **Backend**
  - **Language**: Python 3.x  
  - **Web framework**: Flask  
    - Uses blueprints to organise APIs by domain (e.g., events, registrations, analytics).
  - **ORM / Database access**: SQLAlchemy  
    - Defines ORM models for `User`, `Event`, `EventSession`, `Registration`, `Tag`, etc.
    - Manages sessions and transactions for all persistence operations.
  - **Database**: MySQL 8.x (InnoDB, `utf8mb4`)  
    - Stores all core domain data, including users, events, sessions, registrations, tags, and audience groups.
    - InnoDB provides transactions and row-level locking, which are essential for enforcing per-session capacity under concurrent registrations.
  - **Database driver**: `mysql+pymysql` (via the `pymysql` package).
  - **Configuration**: `.env` + `python-dotenv`  
    - Database URL, debug flags, and other settings are injected via environment variables.
  - **Concurrency experiments**:
    - Python standard library `concurrent.futures.ThreadPoolExecutor` is used to simulate concurrent registration loads and compare different transaction strategies (e.g., with and without `SELECT ... FOR UPDATE`).

- **Frontend**
  - **Framework**: Vue (e.g., Vue 3 + Vite or Vue CLI)  
  - **HTTP client**: Axios (or Fetch API) for calling backend REST APIs and handling JSON responses.
  - **UI components**: A lightweight UI component library such as Element Plus / Ant Design Vue, or custom CSS for forms, tables, and dashboards.
  - **Routing (examples)**:
    - `/` – public event list and filters for visitors.
    - `/events/:id` – event detail and session list.
    - `/my-registrations` – “My registrations” page for visitors.
    - `/admin/events` – staff/admin event and session management.
    - `/admin/analytics` – analytics dashboard.
    - `/checkin` – QR-based check-in panel for on-site staff.

- **Development and tooling**
  - **Testing**: Python `unittest` or `pytest` for unit and integration tests; custom scripts for concurrency/load testing.
  - **Diagramming**: draw.io / Mermaid / similar tools for ER diagrams and system architecture diagrams.
  - **API testing**: Postman / curl / VSCode REST Client.

This technology stack is intentionally lightweight and widely used, focusing on clarity of design and ease of implementation rather than on heavy infrastructure.

---

### 4.3 System Diagram

At a high level, the system consists of three types of clients, a Vue frontend, a layered Flask backend, and a MySQL database.

- **Clients**
  - **Visitor browser**  
    - Accesses the public event browsing and registration UI.
  - **Staff/Admin browser**  
    - Accesses the management dashboards for event/session management and analytics.
  - **Check-in panel browser**  
    - Runs a specialised view that supports QR-code scanning and real-time check-in.

- **Vue Frontend**
  - A single-page (or multi-page) Vue application:
    - Renders the appropriate views for visitors, staff, and the check-in panel.
    - Manages client-side state (e.g., current user, selected event/session).
    - Sends HTTP requests to the backend’s REST APIs via Axios and renders returned data.

- **Flask Backend**
  - A single Flask application, internally divided into:
    - **API / Controllers**: Flask blueprints and route handlers.
    - **Services**: business logic for registration, waiting lists, no-shows, analytics, etc.
    - **Repositories / ORM**: SQLAlchemy models and repository functions for database access.
  - The backend exposes RESTful endpoints such as:
    - `/api/events`, `/api/sessions`, `/api/registrations`, `/api/checkin`, `/api/analytics`, `/api/auth`, etc.

- **MySQL Database**
  - A MySQL instance (InnoDB engine) storing:
    - Users and roles (visitor, staff, admin);
    - Events and sessions (with capacities and waiting list limits);
    - Registrations, check-in timestamps, waiting list statuses;
    - Tags and event-tag relations;
    - Audience group information for analytics (e.g., VIP, Media, Member, General public).

The conceptual system diagram can be illustrated as follows:


![SystemDiagram](SysytemDiagram.png)

*Figure 1. Overall system architecture connecting clients, frontend, backend, and database.*

In this architecture, all user interaction—including the specialised check-in panel—happens through the Vue-based frontend running in a standard web browser. The frontend is responsible for rendering views for visitors, staff, and admins, maintaining client-side state, and issuing HTTP requests to the Flask backend. All communication between frontend and backend uses REST-style APIs with JSON payloads, which keeps the interface simple, language-agnostic, and easy to debug. On the server side, the Flask application delegates domain logic to the Service layer and persists data through the Repository/ORM layer into a MySQL database. The MySQL (InnoDB) engine provides the transactional guarantees and row-level locking required to enforce per-session capacity under concurrent registrations, while also supporting analytical queries for the dashboards.


## 5. Database Design (Track 1)

Below is the **English version of the complete Database Design (Track 1)**, already incorporating your latest requirement:  
- Audience “group” is *per event*,  
- No hard-coded enum for groups,  
- One default group “Member”, other groups are managed by admins.

You can paste this directly into your report and adjust wording/length as needed.

---

## 1. ER Design

### 1.1 Core Entities and Attributes

#### 1. USER

- Meaning: Users of the system, including visitors, staff, and administrators.
- Main attributes:
  - **user_id**: Primary key, identifies a user.
  - **name**: User name.
  - **email**: Login email, unique.
  - **password**: Password (stored securely, e.g., hashed).
  - **role**: User role, one of: visitor / staff / admin.
  - **blocked_until**: Datetime until which the user is not allowed to register (NULL means not blocked).

> Note: We **do not** store group_name on USER anymore. The audience group is managed at the “user + event” level.

---

#### 2. ORGANIZATION (optional)

- Meaning: Organizations that host or operate events.
- Main attributes:
  - **org_id**: Primary key.
  - **org_name**: Organization name, unique.
  - **contact_email**: Contact email (nullable).

---

#### 3. EVENTTYPE

- Meaning: Category of events.
- Main attributes:
  - **type_id**: Primary key.
  - **type_name**: Unique type name, e.g., Exhibition, Concert, Talk, Workshop.

---

#### 4. EVENT

- Meaning: A high-level event project, e.g., “Spring Art Exhibition 2025”.
- Each EVENT can have multiple EVENT_SESSIONs.
- Main attributes:
  - **eid**: Primary key.
  - **org_id**: Foreign key to ORGANIZATION(org_id), nullable.
  - **type_id**: Foreign key to EVENTTYPE(type_id), NOT NULL.
  - **title**: Event title.
  - **description**: Event description (nullable).
  - **location**: Event location.
  - **status**: Event status, one of:
    - draft / published / closed / archived.
  - **created_at**: Creation timestamp.
  - **updated_at**: Last update timestamp.

---

#### 5. EVENT_SESSION

- Meaning: A concrete session of an event, e.g., “20 Dec, 10:00–12:00”.
- One EVENT has multiple EVENT_SESSIONs.
- Main attributes:
  - **session_id**: Primary key.
  - **eid**: Foreign key to EVENT(eid), NOT NULL.
  - **start_time**: Session start time.
  - **end_time**: Session end time.
  - **capacity**: Maximum capacity of the session (capacity > 0).
  - **current_registered**: Current number of successful registrations (current_registered ≥ 0).
  - **waiting_list_limit**: Maximum length of the waiting list (waiting_list_limit ≥ 0).
  - **status**: Session status, one of: open / closed.

---

#### 6. TAG

- Meaning: Tags for events, e.g., Art, Music, Talk, Workshop.
- Main attributes:
  - **tag_id**: Primary key.
  - **tag_name**: Unique tag name.

---

#### 7. EVENT_TAG

- Meaning: Many-to-many relationship between EVENT and TAG.
- Main attributes:
  - **eid**: Foreign key to EVENT(eid).
  - **tag_id**: Foreign key to TAG(tag_id).
- Primary key:
  - Composite primary key (eid, tag_id).

---

#### 8. REGISTRATION

- Meaning: A registration record for a user to a specific session.
- Main attributes:
  - **user_id**: Foreign key to USER(user_id).
  - **session_id**: Foreign key to EVENT_SESSION(session_id).
  - **register_time**: Registration timestamp.
  - **status**: Registration status, one of:
    - registered (successfully registered),
    - waiting (on waiting list),
    - cancelled.
  - **checkin_time**: Check-in timestamp (NULL means not checked in).
  - **queue_position**: Position in the waiting list (NULL if not in waiting status).
- Primary key:
  - Composite primary key (user_id, session_id), ensuring one registration per user per session.

---

#### 9. AUDIENCE_GROUP (new: configurable audience segments)

- Meaning: Configurable audience groups used across events, managed by administrators.
- Examples: Member (default), VIP, Media, Student, etc.
- Main attributes:
  - **group_id**: Primary key.
  - **group_name**: Unique group name, e.g., 'Member', 'VIP'.
  - **description**: Description (nullable).
  - **is_default**: Boolean flag indicating whether this is the system default group (e.g., Member = TRUE).

> The system should provide at least one default group “Member”. Other groups are created and maintained by admins.

---

### 10. EVENT_USER_GROUP (new: user group per event)

- Meaning: Defines which audience group a user belongs to within a specific event.
- Main attributes:
  - **user_id**: Foreign key to USER(user_id).
  - **eid**: Foreign key to EVENT(eid).
  - **group_id**: Foreign key to AUDIENCE_GROUP(group_id).
- Primary key:
  - PRIMARY KEY (user_id, eid)
- Semantics:
  - A user belongs to at most one audience group for a given event.

> Typical flow: When a user registers for any session of an event for the first time, if no special group is specified, the system automatically assigns them to the default group (e.g., Member, where is_default = TRUE).

---

### 1.2 Main Relationships

- **USER – REGISTRATION: one-to-many**
  - A user can have multiple registrations.
  - Each registration belongs to exactly one user.

- **EVENT – EVENT_SESSION: one-to-many**
  - An event can have multiple sessions.
  - Each session belongs to exactly one event.

- **EVENT_SESSION – REGISTRATION: one-to-many**
  - A session can have many registrations.
  - Each registration belongs to exactly one session.

- **EVENT – TAG: many-to-many via EVENT_TAG**
  - An event can have multiple tags.
  - A tag can be used by multiple events.

- **EVENT – EVENTTYPE: many-to-one**
  - Each event has one type.
  - One event type can be used by many events.

- **EVENT – ORGANIZATION: many-to-one**
  - Each event is hosted by one organization.
  - One organization can host many events.

- **AUDIENCE_GROUP – EVENT_USER_GROUP: one-to-many**
  - One audience group can be assigned to many (user, event) pairs.
  - Each EVENT_USER_GROUP entry references exactly one group.

- **USER – EVENT – AUDIENCE_GROUP: ternary relationship (realized by EVENT_USER_GROUP)**
  - EVENT_USER_GROUP(user_id, eid, group_id) describes:
    - “Which group a specific user belongs to within a specific event.”

---

### 1.3 ER Diagram

Conceptually, the database design can be described at two levels. Figure 2 summarises the main entities and relationships in a more abstract, conceptual ER model. Figure 3 refines this view into a physical-level ER / schema diagram that is closer to the actual implementation, highlighting primary keys, foreign keys, and important constraints.

![ERD2](ERdiag2.png)
*Figure 2. Conceptual ER diagram of the event registration domain.*

![ERD1](ERdiag1.png)
*Figure 3. Physical ER / schema diagram with key constraints.*

---

## 2. Schema Design (Tables / Keys / Constraints)

This section describes the main tables, their columns, primary/foreign keys, and important constraints.

### 2.1 USER

- Columns:
  - **user_id** INT, PRIMARY KEY, AUTO_INCREMENT
  - **name** VARCHAR
  - **email** VARCHAR, UNIQUE, NOT NULL
  - **password** VARCHAR, NOT NULL
  - **role** ENUM('visitor','staff','admin'), NOT NULL
  - **blocked_until** DATETIME, NULL
- Constraints:
  - email must be unique.
  - role is restricted to the three predefined values.
  - blocked_until can be NULL (not blocked).

---

### 2.2 ORGANIZATION (optional)

- Columns:
  - **org_id** INT, PRIMARY KEY, AUTO_INCREMENT
  - **org_name** VARCHAR, UNIQUE, NOT NULL
  - **contact_email** VARCHAR, NULL

---

### 2.3 EVENTTYPE

- Columns:
  - **type_id** INT, PRIMARY KEY, AUTO_INCREMENT
  - **type_name** VARCHAR, UNIQUE, NOT NULL

---

### 2.4 EVENT

- Columns:
  - **eid** INT, PRIMARY KEY, AUTO_INCREMENT
  - **org_id** INT, NULL, FOREIGN KEY → ORGANIZATION(org_id)
  - **type_id** INT, NOT NULL, FOREIGN KEY → EVENTTYPE(type_id)
  - **title** VARCHAR, NOT NULL
  - **description** TEXT, NULL
  - **location** VARCHAR, NOT NULL
  - **status** ENUM('draft','published','closed','archived'), NOT NULL
  - **created_at** DATETIME, NOT NULL
  - **updated_at** DATETIME, NOT NULL

---

### 2.5 EVENT_SESSION

- Columns:
  - **session_id** INT, PRIMARY KEY, AUTO_INCREMENT
  - **eid** INT, NOT NULL, FOREIGN KEY → EVENT(eid)
  - **start_time** DATETIME, NOT NULL
  - **end_time** DATETIME, NOT NULL
  - **capacity** INT, NOT NULL, CHECK (capacity > 0)
  - **current_registered** INT, NOT NULL, DEFAULT 0, CHECK (current_registered >= 0)
  - **waiting_list_limit** INT, NOT NULL, DEFAULT 0, CHECK (waiting_list_limit >= 0)
  - **status** ENUM('open','closed'), NOT NULL
- Logical constraints (enforced in application layer):
  - start_time < end_time
  - current_registered ≤ capacity

---

### 2.6 TAG

- Columns:
  - **tag_id** INT, PRIMARY KEY, AUTO_INCREMENT
  - **tag_name** VARCHAR, UNIQUE, NOT NULL

---

## 2.7 EVENT_TAG

- Columns:
  - **eid** INT, NOT NULL, FOREIGN KEY → EVENT(eid)
  - **tag_id** INT, NOT NULL, FOREIGN KEY → TAG(tag_id)
- Primary key:
  - PRIMARY KEY (eid, tag_id)

---

### 2.8 REGISTRATION

- Columns:
  - **user_id** INT, NOT NULL, FOREIGN KEY → USER(user_id)
  - **session_id** INT, NOT NULL, FOREIGN KEY → EVENT_SESSION(session_id)
  - **register_time** DATETIME, NOT NULL
  - **status** ENUM('registered','waiting','cancelled'), NOT NULL
  - **checkin_time** DATETIME, NULL
  - **queue_position** INT, NULL
- Primary key:
  - PRIMARY KEY (user_id, session_id)
- Business semantics:
  - status = 'registered':
    - Counts towards EVENT_SESSION.current_registered.
  - status = 'waiting':
    - On the waiting list, ordered by queue_position.
  - status = 'cancelled':
    - No longer counted.
  - checkin_time IS NOT NULL:
    - Means the user has checked in.
  - “No-show”:
    - A registration with status='registered' and checkin_time IS NULL when the session has finished.

---

### 2.9 AUDIENCE_GROUP

- Columns:
  - **group_id** INT, PRIMARY KEY, AUTO_INCREMENT
  - **group_name** VARCHAR, UNIQUE, NOT NULL
  - **description** VARCHAR, NULL
  - **is_default** BOOLEAN, NOT NULL, DEFAULT FALSE
- Constraints and logic:
  - group_name is unique.
  - There should be at least one row with is_default = TRUE, for example “Member”.
  - Admins can create, edit, or delete audience groups through the admin UI.

---

### 2.10 EVENT_USER_GROUP

- Columns:
  - **user_id** INT, NOT NULL, FOREIGN KEY → USER(user_id)
  - **eid** INT, NOT NULL, FOREIGN KEY → EVENT(eid)
  - **group_id** INT, NOT NULL, FOREIGN KEY → AUDIENCE_GROUP(group_id)
- Primary key:
  - PRIMARY KEY (user_id, eid)
- Semantics:
  - Each user has at most one audience group per event.
  - When a user participates in an event for the first time, if no explicit group is given, the system inserts a row mapping them to the default group (e.g., Member).

---

## 3. Normalization Level

Overall, the schema satisfies at least **Third Normal Form (3NF)** and is close to BCNF.

### 3.1 First Normal Form (1NF)

- All columns hold atomic values, no repeated groups or arrays.
- Examples:
  - TAG uses one row per tag_name; there is no “comma-separated list of tags”.
  - EVENT_TAG is a separate table for the many-to-many relationship between EVENT and TAG.
  - EVENT_USER_GROUP holds one row per (user, event, group) association, no multi-valued group fields.

---

### 3.2 Second Normal Form (2NF)

- For tables with a single-column primary key (USER, EVENT, EVENT_SESSION, TAG, ORGANIZATION, EVENTTYPE, AUDIENCE_GROUP):
  - All non-key attributes depend fully on the primary key.
- For tables with composite primary keys:
  - REGISTRATION(user_id, session_id):
    - register_time, status, checkin_time, queue_position all depend on the combination (user_id, session_id), not on one part alone.
  - EVENT_TAG(eid, tag_id):
    - Only represents the association; there are no additional non-key attributes.
  - EVENT_USER_GROUP(user_id, eid):
    - group_id depends on the combination (user_id, eid).

---

### 3.3 Third Normal Form (3NF)

- All non-key attributes depend directly on the key and not on other non-key attributes (no transitive dependencies).
- Examples:
  - EVENTTYPE, ORGANIZATION, TAG, and AUDIENCE_GROUP are modeled as separate tables, referenced by IDs, instead of duplicating type_name, org_name, or group_name in multiple places.
  - User role (role) is stored in USER and not duplicated in REGISTRATION or EVENT.
  - Audience group definitions (group_name) are stored in AUDIENCE_GROUP and linked via EVENT_USER_GROUP. They are not duplicated in USER or REGISTRATION.
- This design reduces redundancy and update anomalies while keeping the schema clean and maintainable.

---

## 4. Indexing Strategy

We consider both **OLTP** (transactional) queries and **analytics** queries.

### 4.1 Primary Key Indexes

All primary keys have corresponding indexes, e.g.:

- USER(user_id)
- ORGANIZATION(org_id)
- EVENTTYPE(type_id)
- EVENT(eid)
- EVENT_SESSION(session_id)
- TAG(tag_id)
- AUDIENCE_GROUP(group_id)
- EVENT_TAG(eid, tag_id)
- REGISTRATION(user_id, session_id)
- EVENT_USER_GROUP(user_id, eid)

---

### 4.2 Indexes for Common Access Paths

#### 4.2.1 USER

- UNIQUE(email):
  - For login and duplicate-checking.
- INDEX(role):
  - For listing all staff/admin users in back-office.
- INDEX(blocked_until):
  - For checking whether a user is currently blocked when they attempt to register.

---

#### 4.2.2 EVENT

- INDEX(type_id, status):
  - To list events by type and status (e.g., published exhibitions only).
- INDEX(created_at) or INDEX(updated_at):
  - For sorting events by time and simple time-based queries.

---

#### 4.2.3 EVENT_SESSION

- INDEX(eid):
  - To quickly fetch all sessions for a given event.
- INDEX(start_time) / INDEX(end_time):
  - For time-range queries, such as “sessions today” or “sessions next week”.
- INDEX(status):
  - To filter open sessions (status='open') efficiently.

---

#### 4.2.4 REGISTRATION

- Primary key: (user_id, session_id).
- INDEX(user_id):
  - For listing a user’s registration history.
- INDEX(session_id):
  - For listing all registrations for a specific session (e.g., check-in page).
- Composite indexes:
  - INDEX(session_id, status):
    - For fetching registered / waiting users of a given session.
  - INDEX(user_id, status) (optional):
    - For per-user statistics by status (e.g., counting no-shows).
- INDEX(checkin_time):
  - For aggregations based on check-in date (e.g., “check-ins today”).

---

#### 4.2.5 EVENT_TAG

- Primary key: (eid, tag_id).
- INDEX(tag_id):
  - For searching events by tag, especially when joining EVENT_TAG and EVENT.

---

#### 4.2.6 AUDIENCE_GROUP / EVENT_USER_GROUP

- AUDIENCE_GROUP:
  - UNIQUE(group_name).
  - INDEX(is_default):
    - To quickly find the default group (e.g., Member) when assigning groups.
- EVENT_USER_GROUP:
  - PK(user_id, eid).
  - INDEX(eid, group_id):
    - For statistics per event and audience group.
  - INDEX(user_id):
    - For analyzing which groups a user belongs to across events.

---

### 4.3 Analytics-Oriented Indexing

- Counting check-ins by date:
  - Relies on REGISTRATION.checkin_time. Indexing this column greatly speeds up date-range queries.
- Participation statistics by audience group and event type:
  - Involves JOINs between USER, REGISTRATION, EVENT_SESSION, EVENT, EVENTTYPE, EVENT_USER_GROUP, and AUDIENCE_GROUP.
  - Indexes on EVENT.type_id, AUDIENCE_GROUP.group_name, EVENT_USER_GROUP.group_id, and REGISTRATION.status can significantly improve performance.
- Overall:
  - The schema stays normalized (3NF) for data consistency, and we rely on indexes to achieve acceptable performance for both OLTP and reporting queries.

---

## 5. Sample Queries

This section illustrates typical business queries, no-show / penalty logic, and analytical queries involving audience groups and tags.

### 5.1 List all open sessions on a specific day with registration counts

Conditions:
- EVENT.status = 'published'
- EVENT_SESSION.status = 'open'
- start_time is on a specific date (here: today).

````markdown
```sql
SELECT
    e.eid,
    e.title,
    s.session_id,
    s.start_time,
    s.end_time,
    s.capacity,
    s.current_registered
FROM EVENT e
JOIN EVENT_SESSION s ON e.eid = s.eid
WHERE e.status = 'published'
  AND s.status = 'open'
  AND DATE(s.start_time) = CURDATE();
```
````

---

### 5.2 Count user no-shows in the last 30 days

Definition of no-show:  
The session has ended, REGISTRATION.status = 'registered', and checkin_time IS NULL.

````markdown
```sql
SELECT COUNT(*) AS no_show_count
FROM REGISTRATION r
JOIN EVENT_SESSION s ON r.session_id = s.session_id
WHERE r.user_id = :uid
  AND r.status = 'registered'
  AND r.checkin_time IS NULL
  AND s.end_time >= (NOW() - INTERVAL 30 DAY)
  AND s.end_time < NOW();
```
````

> The application can use no_show_count to update USER.blocked_until and implement a penalty/blocking mechanism.

---

### 5.3 Search published events by tag

````markdown
```sql
SELECT DISTINCT e.*
FROM EVENT e
JOIN EVENT_TAG et ON e.eid = et.eid
JOIN TAG t ON et.tag_id = t.tag_id
WHERE e.status = 'published'
  AND t.tag_name = :tagName
ORDER BY e.created_at DESC;
```
````

---

### 5.4 Total registered users for sessions today

````markdown
```sql
SELECT
    DATE(s.start_time) AS day,
    COUNT(*) AS total_registrations
FROM REGISTRATION r
JOIN EVENT_SESSION s ON r.session_id = s.session_id
WHERE r.status = 'registered'
  AND DATE(s.start_time) = CURDATE()
GROUP BY DATE(s.start_time);
```
````

---

### 5.5 Total check-ins today

````markdown
```sql
SELECT
    DATE(r.checkin_time) AS day,
    COUNT(*) AS total_checkins
FROM REGISTRATION r
WHERE r.checkin_time IS NOT NULL
  AND DATE(r.checkin_time) = CURDATE()
GROUP BY DATE(r.checkin_time);
```
````

---

### 5.6 Participation by audience group and event type (using AUDIENCE_GROUP)

Example: Count registered participants per audience group for a given event type.

````markdown
```sql
SELECT
    g.group_name,
    COUNT(*) AS registrations
FROM REGISTRATION r
JOIN EVENT_SESSION s ON r.session_id = s.session_id
JOIN EVENT e ON s.eid = e.eid
JOIN EVENTTYPE et ON e.type_id = et.type_id
JOIN EVENT_USER_GROUP ug
    ON ug.user_id = r.user_id
   AND ug.eid = e.eid
JOIN AUDIENCE_GROUP g ON ug.group_id = g.group_id
WHERE r.status = 'registered'
  AND et.type_name = :typeName
GROUP BY g.group_name
ORDER BY registrations DESC;
```
````

> To count only checked-in participants, add:
> - AND r.checkin_time IS NOT NULL

## 3. Implementation Overview

The system is implemented as a monolithic Flask application with a clear internal layering. The Flask app exposes RESTful JSON APIs under `/api/*`, organised into blueprints for authentication, events, registration, analytics, and check-in. Each API handler delegates to a corresponding service module that encapsulates business rules such as capacity checks, waiting-list promotion, no-show penalties, and analytics aggregation. The service layer, in turn, calls repositories and ORM models for database access. For write-heavy transactional flows it uses raw SQL via PyMySQL, while for analytics and reporting queries it uses SQLAlchemy ORM sessions and models.

The frontend is implemented as a Vue-based single-page application. It communicates with the backend exclusively via HTTPS and JSON, using standard HTTP verbs and status codes. The main screens include: a public event list and details page, a registration flow with waiting list handling, a “My registrations” page, a check-in view that consumes QR codes, and a simple analytics dashboard for staff or administrators. All user-visible state transitions (e.g., moving from waiting to registered, or applying penalties after repeated no-shows) are triggered via backend APIs that enforce the same consistency rules regardless of which client is used.

The database is a MySQL 8.x instance using the InnoDB engine, with the schema designed and normalised as described in Section 2. The implementation closely follows the ER and schema diagrams (Figures 2 and 3), with additional indexes and constraints added via migrations and initialisation scripts. Data initialisation and demo data are provided through Python scripts so that the environment can be recreated in a repeatable way for development, testing, and evaluation.

## 4. Testing and Evaluation

### 4.1 Automated testing

For this project we adopted an automated testing strategy based on `pytest`. The backend includes smoke tests that verify the service is running and that key health endpoints (such as `/` and `/api/events/health`) return the expected JSON payloads and HTTP status codes. Unit tests focus on core service functions, for example the search module’s date parsing and error-handling logic, ensuring that invalid inputs and lower-level database failures are handled gracefully and reported through domain-specific exceptions rather than leaking internal errors to the client.

Beyond simple unit and smoke tests, the test suite also contains integration tests that exercise full API flows against a real MySQL instance using the same schema as in production. These tests create events and sessions, perform registrations and cancellations, and verify that capacity tracking, waiting-list insertion and promotion, and QR-code based check-in are all reflected correctly in the database. Concurrency tests simulate a large number of users registering for the same session concurrently (using Python thread pools), and assert that no more than the configured capacity can be registered, that waiting-list positions remain consistent, and that there are no duplicate registrations per user per session.

Performance-oriented tests focus on ensuring that typical query paths—such as listing published events, searching by tag or keyword, or loading basic analytics—complete within a reasonable time budget (for example, less than one second on the development machine). These tests complement the indexing strategy described in Section 4 of the database design, and they help validate that the mix of OLTP and analytical queries can be served efficiently without resorting to premature denormalisation.

### 4.2 Usability evaluation

In addition to automated tests, we conducted a small but systematic usability evaluation to verify that the system behaves as expected from an end-user perspective. The evaluation was documented in a “Usability Test Matrix” (in the DevNote), which defines realistic scenarios such as browsing published events, registering for an open session, being placed on a waiting list when a session is full, cancelling a registration to free a seat, scanning a QR code at the venue to check in, blocking registration for users with repeated no-shows, and loading a simple analytics dashboard.

For each scenario, the matrix specifies the purpose, the concrete steps that a user or staff member should follow, the key metrics or checks (for example, correct status transitions, accurate counters, and response times within one second), and the expected result. The tests were executed by 夏晋, and all scenarios passed successfully. In practice this means that a visitor can discover events, complete registrations (or be placed on the waiting list if needed), receive a QR code, and check in at the venue without encountering inconsistent states; staff members can cancel registrations, monitor attendance, and review basic statistics; and the system enforces penalty rules for repeated no-shows. Taken together, the automated regression tests and the usability evaluation provide reasonable confidence that the system is both functionally correct and usable for the intended scale of a public event registration and check-in service.

---
# 这里是重复段落，补充了最后一点

Below is a cleaned-up, **Track 1–oriented, all‑English** version of Sections 1–7, with:

- Correct numbering that matches the Track 1 outline.
- A complete, non‑abbreviated Contribution Statement.
- A new **“Challenges and Lessons Learned”** section, incorporating and slightly extending the points you mentioned.

You can paste this into your `XXXX-report.pdf` and then keep Sections 8–9 (References, Links) below it.

---

## 1. Group Information

- (Temporarily omitted here; please fill in the full names and undergraduate majors of all group members according to the course requirement.)

---

## 2. Contribution Statement

Each team member contributed to the project in clearly defined roles, including requirements clarification, architecture and database design, backend implementation, frontend implementation, testing, documentation, and presentation. The following table summarises the main responsibilities and concrete deliverables of each member.

| Member       | Main Contributions                                                                                                                      |
|--------------|------------------------------------------------------------------------------------------------------------------------------------------|
| Zhou Linquan | Led the overall project architecture design; designed the main system workflow and separation of concerns; implemented core backend–frontend integration; supported both backend and frontend troubleshooting; coordinated technical decisions and resolved integration issues. |
| Li Yuhang    | Implemented key frontend views using Vue, including event browsing, registration flows, and basic management pages; contributed significantly to the written report and visual documentation (figures, diagrams); helped refine user interface details and interaction flows from a usability perspective. |
| Tan Cong     | Implemented major backend business logic, including registration and waiting-list handling, no-show penalty rules, and analytics endpoints; designed and implemented core parts of the database schema (tables, relationships, constraints); participated in performance and concurrency-related tests against the MySQL database. |
| Huang Kaisi  | Implemented backend REST APIs for events, sessions, registrations, and check-in; contributed to database implementation details such as indexes and integrity constraints; assisted in debugging complex query logic and ensuring consistency between the ORM models and the physical schema. |
| Xia Jin      | Designed and executed automated tests and manual usability scenarios; implemented and ran pytest-based unit and integration tests; documented the usability test matrix and test results; contributed to report writing, including the testing and evaluation sections, and helped ensure coherence and consistency of the final document. |

Overall, all members participated in requirements discussions and periodic design reviews, and jointly reviewed the final implementation and report.

---

## 3. Project Overview

### 3.1 Context and Problem Statement

This project targets exhibition and performance spaces, such as public art venues, cultural centres, community theatres, library event areas, and science exhibition halls. These venues frequently host various public, non-commercial events, including talks, screenings, workshops, small performances, and themed exhibitions.

In practice, event registration and on-site check-in in such venues often suffer from several issues:

- **Fragmented tools and siloed data**  
  Many venues rely on a mixture of paper forms, spreadsheet files, and ad‑hoc online forms. Data for different events and sessions lives in different files or platforms. After events end, answering questions such as “which audience segments attended which types of events?” requires significant manual consolidation and cleaning.

- **Unreliable capacity management**  
  For popular events or small venues, staff need to enforce strict capacity limits. Without a unified system, manual counting is error‑prone: it is easy to over‑book seats or fail to detect duplicated registrations. A typical problem in public venues is that some people register early to “reserve” seats but never show up, while others who genuinely want to attend cannot register because the session is already full.

- **Inefficient on‑site check‑in**  
  Staff often check paper lists or manually search exported spreadsheets to verify attendees. This leads to long queues, mis‑checks, and an overall poor experience for both staff and visitors.

- **Limited analytics and service optimisation**  
  Venues may want to know, for example: which types of events attract which audience groups; how “members”, “media”, and general visitors differ in attendance behaviour; and what the typical attendance rates are for different time slots or themes. Due to fragmented data, these questions are hard to answer in a systematic way.

Our system aims to address these issues in a public, non‑commercial context, focusing on fair allocation of limited seats and data‑driven service improvement, rather than on ticket sales or payment processing.

---

### 3.2 System Goals and Positioning

Based on the above context, we design and implement an event registration and check‑in system for exhibition and performance spaces with the following goals:

1. **Unified management of multiple events and sessions**  
   Model events that may span multiple days, each consisting of multiple sessions (time slots). Each session has its own start and end time and capacity.

2. **Strict capacity control and fair allocation without payments**  
   The system does not handle any ticket prices, payments, or refunds. All events are assumed to be free to attend. Instead of financial logic, the system focuses on:
   - Per‑session capacity limits;
   - Waiting lists for popular sessions;
   - A no‑show penalty mechanism.  
   Together, these aim to make seat allocation more orderly and fair in a public‑resource setting.

3. **Improved resource utilisation through waiting lists and no‑show penalties**  
   When a session is full, the system maintains a waiting list (typically first‑in‑first‑out). If a registered user cancels, waiting users can be automatically or semi‑automatically promoted to “registered”. For visitors who repeatedly register but do not show up, the system tracks no‑show occurrences. If a user accumulates three no‑shows within 30 days, their ability to register for new sessions is blocked for one month. The goal is not financial punishment but to reduce wasted seats and to give genuinely interested visitors a better chance of attending.

4. **Efficient QR‑code‑based check‑in**  
   After successful registration, the system generates a digital ticket containing a QR code whose payload is a JSON object encoding the user and registration information. On‑site staff use a dedicated check‑in panel to scan these codes, verify the registration, and mark the attendee as checked‑in, reducing bottlenecks and manual search.

5. **Operator‑facing analytics and export for further analysis**  
   The system provides an analytics panel for staff to view daily totals of registrations and check‑ins, to analyse participation by event type or tag and by audience group (for example, VIP, Media, Member, General public), and to export per‑event or per‑session data to CSV for further analysis in external tools.

---

### 3.3 Target Users and Roles

The application distinguishes three main roles:

1. **Visitor**  
   Browses upcoming events and sessions, filters events by date or tag, registers for sessions and cancels registrations, obtains a QR‑code ticket for on‑site check‑in, and is subject to the no‑show policy.

2. **Staff**  
   Uses the back‑office interface to create, edit, and archive events and sessions; manage tags and audience groups; view registration and attendance lists; force‑add attendees (for example, VIPs, media, on‑site registrations) regardless of normal capacity; and operate the check‑in panel.

3. **Admin**  
   A high‑privilege role that manages staff accounts, global configuration, and system integrity, and performs low‑level maintenance and corrective operations when necessary.

---

### 3.4 Core Features and Characteristics

The system provides several core functional modules:

1. **Event and session management**  
   Events can contain multiple sessions, each with start and end time, capacity, current registration count, waiting list limit, and status. Events have a lifecycle of draft, published, closed, and archived.

2. **Registration and waiting list management**  
   When a visitor attempts to register, the system either creates a registered record and increments the session’s counter or, if the session is full, places the visitor in a waiting list (if enabled) or rejects the registration as “full”. When a registered user cancels, capacity is freed and can be allocated to the earliest waiting user.

3. **QR‑code‑based check‑in and check‑in panel**  
   The system generates ticket‑style QR codes with JSON payloads containing user and session identifiers. The check‑in panel scans codes, validates registrations, records check‑in timestamps, and displays key information for staff verification. Check‑in operations are idempotent to avoid double counting under repeated scans.

4. **No‑show tracking and registration blocking**  
   A no‑show is defined as a registration that remains in “registered” status without a check‑in after the session has ended. When a user attempts to register, the system checks their recent no‑show count and may block new registrations if the penalty threshold is exceeded.

5. **Analytics and CSV export**  
   Staff can access aggregate dashboards for registrations and check‑ins and can export detailed event or session data as CSV files for external analysis.

---

### 3.5 Technical Focus: Concurrency and Data Integrity

Although the expected concurrency level is moderate (for example, around one hundred concurrent users at peak), the project intentionally focuses on transaction management and concurrency control:

- The relational schema separates events, sessions, and registrations, with a maintained `current_registered` field per session.
- During registration, database transactions and row‑level locks (for example, `SELECT … FOR UPDATE`) are used to ensure that the number of registered users never exceeds the configured capacity and that counters remain consistent.
- Micro‑benchmarks simulate multiple threads registering for the same session to confirm experimentally that over‑booking does not occur.

---

### 3.6 Open‑Source Orientation and Reusability

The system is designed as an open‑source, reusable infrastructure component for public and cultural venues, rather than as a venue‑specific or commercial ticketing platform. It is decoupled from payment providers and commercial ticketing services, assumes free events by default, and can be deployed in various venues (galleries, cultural centres, libraries, universities) with minimal customisation, mainly through configuration of audience groups and user interface themes. The database schema and APIs are intentionally modular to allow future extensions such as more advanced role‑based access control, recommendation features, or deeper integration with analytical platforms.

---

## 4. Technical Design

### 4.1 Software Architecture

We adopt a monolithic web application architecture with a clear front‑end / back‑end separation and a layered, Model–View–Controller‑style structure on the backend.

- **Monolithic backend service**  
  All server‑side functionality is implemented in a single Flask application. This simplifies deployment and configuration and is sufficient to demonstrate database design, transaction management, and concurrency control.

- **Front‑end / back‑end separation**  
  A Vue‑based frontend running in the browser interacts with a Flask‑based backend through RESTful HTTP interfaces with JSON payloads. The frontend is responsible for all user interactions and rendering, while the backend exposes APIs for data access and business operations.

- **Layered backend design**  
  Inside the Flask application we distinguish:
  1. **API / Controller layer** – Flask routes and blueprints that receive HTTP requests, validate input, delegate to services, and return JSON responses with appropriate status codes.  
  2. **Service (business logic) layer** – Implements registration and cancellation flows, waiting‑list management, no‑show detection and blocking, analytics aggregation, and CSV export.  
  3. **Data access / Repository layer** – Wraps SQLAlchemy ORM models and raw SQL where necessary, providing repository functions and controlling sessions and transactions.  
  4. **Database layer** – A MySQL (InnoDB) database that stores persistent data and enforces integrity via primary keys, foreign keys, and constraints, and that provides transactionality and row‑level locking.

---

### 4.2 Technology Stack

- **Backend**  
  - Programming language: Python 3.x  
  - Web framework: Flask  
  - ORM and database access: SQLAlchemy plus PyMySQL  
  - Database: MySQL 8.x (InnoDB, `utf8mb4`)  
  - Configuration: environment variables loaded via `python-dotenv`  
  - Concurrency experiments: `concurrent.futures.ThreadPoolExecutor` for simulating concurrent registration loads

- **Frontend**  
  - Framework: Vue (for example, Vue 3 with Vite)  
  - HTTP client: Axios or Fetch API  
  - UI components: lightweight component library (such as Element Plus or Ant Design Vue) plus custom styling  
  - Typical routes: public event list, event detail, “my registrations”, admin event management, analytics dashboard, and a dedicated `/checkin` route for QR‑based check‑in

- **Tooling**  
  - Testing: `pytest` for unit, integration, and concurrency tests  
  - Diagramming: tools such as draw.io or Mermaid for system and ER diagrams  
  - API testing: Postman, curl, or the Visual Studio Code REST Client

---

### 4.3 System Diagram

At a high level the system consists of three types of clients (visitor, staff/admin, and check‑in panel), a Vue frontend, a layered Flask backend, and a MySQL database. All user interaction happens through web browsers, and all communication between the frontend and backend uses JSON‑based REST APIs over HTTP(S). The MySQL database serves as the single source of truth and provides transactional guarantees, while the analytics queries are supported by appropriate indexing.

(See Figure 1 in the full report for the detailed system diagram.)

---

## 5. Database Design (Track 1)

The database design for Track 1 includes entities for users, organisations, event types, events, sessions, tags, registrations, audience groups, and per‑event audience group assignments, with relationships and constraints described through an ER model and a normalised schema. The schema is designed to satisfy at least Third Normal Form, reducing redundancy while supporting both transactional and analytical workloads. It also includes an indexing strategy and sample SQL queries for common access paths, no‑show logic, and analytics by audience group and event type.

(Full details, including table definitions, keys, constraints, indexes, and sample queries, follow in the database design section of the report.)

---

## 6. Evaluation and Results (Track 1)

The evaluation for Track 1 focuses on query correctness, efficiency, and usability of the registration and check‑in flows.

- **Functional and integration correctness**  
  Automated tests and manual scenarios verify that registrations respect capacity limits, waiting‑list promotion works correctly, no‑show penalties are enforced, and QR‑code check‑in updates the database consistently. Integration tests exercise full API flows against a real MySQL instance using the same schema as in production.

- **Concurrency and capacity enforcement**  
  Micro‑benchmarks simulate many concurrent registration attempts for sessions with limited capacity. The results confirm that, under transaction and locking settings, the number of registered users never exceeds the configured capacity and that waiting‑list positions remain consistent. No duplicate registrations per user per session were observed.

- **Query performance and analytics responsiveness**  
  Performance‑oriented tests measure the response times of typical query paths such as listing published events, searching by tag, loading per‑day registration and check‑in statistics, and computing participation by audience group. For the tested dataset size, these queries complete within an acceptable latency budget (for example, within one second on the development environment), validating the indexing strategy.

- **Usability evaluation**  
  A small but systematic usability evaluation, documented in a “Usability Test Matrix”, covers realistic scenarios such as browsing published events, registering for open sessions, being placed on a waiting list, cancelling to free a seat, scanning QR codes at the venue, blocking users with repeated no‑shows, and loading the analytics dashboard. All defined scenarios were executed and passed, indicating that the system is usable for the intended context and scale.

---

## 7. Challenges and Lessons Learned

### 7.1 Challenges

During the project the team encountered several practical challenges:

1. **Time management and planning**  
   The team did not always manage time effectively. Some key tasks (such as writing automated tests and polishing the user interface) were postponed until late stages of the project. This resulted in a compressed schedule before the deadline and limited opportunities to refactor or explore alternative designs.

2. **Team communication and collaboration**  
   Communication within the team was not always smooth. At times requirements and design decisions were not fully aligned across members, leading to duplicated work and integration conflicts, especially between frontend and backend interfaces. Regular stand‑up style meetings and clearer task assignment would have reduced misunderstandings.

3. **Collaboration workflow and coordination**  
   Although GitHub was used for version control, the team needed time to adapt to a more disciplined workflow. For example, merge conflicts occurred when branches were not synchronised frequently enough, and some pull requests lacked detailed descriptions or code reviews. This slowed down progress when multiple members were working on related modules.

4. **Containerisation and deployment attempts with Docker**  
   The team attempted to use Docker and Docker Compose to simplify deployment of the backend, frontend, and database. However, repeated failures in writing a correct `docker-compose.yml` file (for example, incorrect service dependencies, environment configuration, and networking between containers) led to a significant time cost without a stable outcome. Due to time pressure, the team decided to fall back to a more manual deployment approach and to leave a robust Docker‑based deployment as future work.

5. **Balancing functionality, performance, and simplicity**  
   The project needed to demonstrate non‑trivial concurrency control and analytics while remaining implementable within the course timeline. Designing the database and APIs so that they were both conceptually clean and efficient in practice required several iterations and trade‑offs, especially for no‑show tracking and audience group management.

---

### 7.2 Lessons Learned

Despite these challenges, the team gained several important lessons and practical skills:

1. **Using GitHub for collaborative software development**  
   The team learned to use Git and GitHub for collaborative coding: creating branches for features, committing frequently, opening pull requests, resolving merge conflicts, and reviewing code. This experience highlighted the importance of clear commit messages, branch naming conventions, and basic continuous integration to maintain code quality.

2. **Understanding software architecture and REST‑based front‑end / back‑end separation**  
   The project provided concrete experience with a layered software architecture: a Vue frontend calling REST APIs implemented in a Flask backend, which in turn interacts with a relational database. The team gained a deeper understanding of separation of concerns, API design, stateless communication, and how architectural decisions influence maintainability and scalability.

3. **Implementing a backend using Python and Flask**  
   The team became more familiar with building backend services in Python, including defining REST endpoints in Flask, managing request validation and error handling, organising code into controllers, services, and repositories, and working with SQLAlchemy ORM and raw SQL when necessary. The importance of transactions and row‑level locking for data integrity became much clearer through hands‑on implementation.

4. **Designing and normalising a relational database schema**  
   Through the iterative design of the schema, the team learned how to model entities and relationships, apply normalisation (up to at least Third Normal Form), define primary and foreign keys, and design indexes for both transactional and analytical queries. Modelling configurable audience groups and per‑event group assignments was a concrete example of translating requirements into a clean schema.

5. **Writing automated tests and designing usability scenarios**  
   Implementing pytest‑based unit and integration tests showed the value of automated regression testing. At the same time, designing a usability test matrix demonstrated that non‑functional aspects—such as user experience, clarity of flows, and response time—are as important as pure functional correctness.

6. **Appreciating the importance of planning and communication**  
   Finally, the difficulties with time management, communication, and Docker‑based deployment underlined that software engineering is not only about coding. Early planning, realistic scoping, clear communication, and incremental integration are crucial for delivering a reliable system within limited time and resources.

These challenges and lessons significantly shaped how the team understands software engineering practice and will influence how future projects are planned, implemented, and evaluated.